In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
from nnAudio.Spectrogram import CQT1992v2
from torch.utils.data import DataLoader, Dataset
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv(INPUT_PATH / "training_labels.csv")

files = list((INPUT_PATH / "train").rglob("*.npy"))
FILE_PATH_DICT = {x.stem: str(x) for x in files}
df["path"] = df["id"].apply(lambda x: FILE_PATH_DICT[x])

In [ ]:
PARAMS = {
    "sr": 2048,
    "fmin": 30,
    "fmax": 400,
    "hop_length": 64,  # count of signal points used for 1 tranformed point
#     "bins_per_octave": 32,  # y-axe of transformed image depends from this parameter, for example [91,] if bins=16 and 42 if bins=8
}
INPUT_PATH = Path("/home/trytolose/rinat/kaggle/grav_waves_detection/input")

In [ ]:
cqt = CQT1992v2(**PARAMS) 

def apply_qtransform(waves):
#     waves = np.hstack(waves)
    waves = waves / np.max(waves)
    waves = torch.from_numpy(waves).float()

    image = cqt(waves).squeeze().numpy()
    return image


In [ ]:
df.head()

In [ ]:
for _, row in df[df['target']==1].sample(10, replace=False).iterrows():
    wave = np.load(row["path"])
    xx = apply_qtransform(wave)
    print(xx.shape)
    plt.figure(figsize=(20,20))
    plt.imshow(xx[2])
    plt.title(row['target'])
    plt.show()
    break

In [ ]:
from typing import List

In [ ]:
def longestSubarray(nums: List[int]) -> int:
    ones_count_best = 0
    curr = 0
    for i in range(len(nums)):
        if nums[i] == 1:
            curr += 1
        elif nums[i] == 0 and nums[i+1] == 1:
            curr += 1
        else:
            
            if curr > ones_count_best:
                ones_count_best = curr
            curr = 0
        if i == len(nums)-1:
            if curr > ones_count_best:
                ones_count_best = curr
            
        print(curr)
    return ones_count_best

In [ ]:
longestSubarray([1,1,0,1])